In [1]:
# creates: band-alignment.png
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from ase.db import connect
import ase
from pymatgen.core import Lattice, Structure, Molecule

In [ ]:
atom_dict1=['na','H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Xe','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']
#removing nobels 
atom_dict=['na','H','Li','Be','B','C','N','O','F','Na','Mg','Al','Si','P','S','Cl','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Cs','Ba','La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr','Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn','Nh','Fl','Mc','Lv','Ts','Og']

In [ ]:
#Encode Voxel images from C2DB database
def pd(i,j,k):
    posx= i*(12/64)
    posy= j*(12/64)
    posz= k*(12/64)
    return np.array([posx, posy, posz])
#Change to give atomic positions directly
db_dir='c2db.db'
db = ase.db.connect(db_dir)
counter=0
# Select elements according to the select variable
rows = db.select('gap>=0')
xproto = []  # prototype

# get the data from C2DB database
for row in rows:
    if counter<100:
        # Get the class for selection
        try:
            m_class = row.get("class")
        except:
            m_class = ""
        # Get the stability for selection
        try:
            ds_level = row.dynamic_stability_level
        except:
            ds_level = 0

        # Filter materials going to be used for training        

        cmp=np.zeros((79,64,64,64))
        num=np.zeros((79))
        brd=0.25

        if row.natoms <= 10:
            structure = Structure(np.array(row.cell), row.symbols, row.positions,coords_are_cartesian=True)
            structure.to(filename=str(row.formula)+'.cif')
            print(row.formula)
            xform = []  # formulas (will be used to generate onehotvector of elements)
            xnumbat = []  # number of atoms
            x_target = []  # target value, defined by the parameter target.
            newpos=np.transpose(np.array([np.transpose(row.positions)[1],np.transpose(row.positions)[0],np.transpose(row.positions)[2]]))
            for h in range(row.natoms):
                k=(atom_dict.index(row.symbols[h]))
                minx=np.min(np.min(np.transpose(newpos)[0]))
                miny=np.min(np.min(np.transpose(newpos)[1]))
                minz=np.min(np.min(np.transpose(newpos)[2]))
                atompos=np.array(newpos[h])-np.array([minx,miny,minz])
                xform.append(k)
                x_target.append(atompos)
                temp=np.zeros((64,64,64))
                for i in range(64):
                    for j in range(64):
                        for l in range(64):
                            pos= pd(i,j,l)
                            r=np.linalg.norm(pos-atompos)
                            temp[i,j,l]=np.exp(-(r**2)/(2*brd))
                temp=temp/np.max(temp)
                cmp[k,:,:,:]=cmp[k,:,:,:]+temp
            for j in range(79):
                if np.max(cmp[j,:,:,:])>0:
                    cmp[j,:,:,:]=cmp[j,:,:,:]/np.max(cmp[j,:,:,:])
            cmp=np.expand_dims(np.array(cmp),4)
            #Uncomment if saving encoded vectors, otherwise save full Voxel image
            #cmp=atomencoder.predict(cmp)
            xproto.append(cmp)
        counter=counter+1




In [ ]:
# Encode nit Cell information
db_dir='c2db.db'
db = ase.db.connect(db_dir)
# Select elements according to the select variable
rows = db.select('gap>=0')
xprotol = []  # prototype
xform = []  # formulas (will be used to generate onehotvector of elements)
xnumbat = []  # number of atoms
x_target = []  # target value, defined by the parameter target.
counter=0
for row in rows:
    if counter <100:
        # Get the class for selection
        try:
            m_class = row.get("class")
        except:
            m_class = ""
        # Get the stability for selection
        try:
            ds_level = row.dynamic_stability_level
        except:
            ds_level = 0
        # Filter materials going to be used for training       
        cmp1=np.zeros((32,32,32))
        brd=1
        if row.natoms <= 10 and counter==78:
            xform.append(row.formula)
            xnumbat.append(row.natoms)
            temp=[]
            for i in range(32):
                for j in range(32):
                    for l in range(32):
                            vec=(row.cell-15*np.sign(row.cell[2:,2:])*np.array([[0,0,0],[0,0,0],[0,0,1]]))
                            CL=np.array([vec[1],vec[0],vec[2]])
                            a=np.dot(np.array([i/31,0,0]),CL)
                            b=np.dot(np.array([0,j/31,0]),CL)
                            c=np.dot(np.array([0,0,l/31]),CL)
                            ab=np.dot(a,b)
                            ac=np.dot(a,c)
                            bc=np.dot(b,c)
                            pos=(np.linalg.norm(a)+np.linalg.norm(b)+np.linalg.norm(c)+ab+ac+bc) 
                            r=np.linalg.norm(pos)
                            cmp1[i,j,l]=np.exp(-(np.abs(r))/(2*brd))
    
            temp.append(cmp1)
            temp=np.expand_dims(np.array(temp),4)
            #Uncomment if saving the enocded lattice image
            #temp=xprotol.append(latencoder.predict(temp))
            xprotol.apppend(temp)
            x_target.append(row.cell)
        #print(counter)
        counter=counter+1